In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv(r'../input/big-mart-sales-prediction/train_v9rqX0R.csv')
test = pd.read_csv(r'../input/big-mart-sales-prediction/test_AbJTz2l.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test],ignore_index=True)

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.head()

In [ ]:
data.nunique()

In [ ]:
data.isnull().sum()

In [ ]:
data['Item_Weight'].fillna(data['Item_Weight'].mean(),inplace=True)

In [ ]:
data['Outlet_Size'].value_counts()

In [ ]:
sns.countplot(x='Outlet_Size',data=data)
plt.show()

In [ ]:
data['Outlet_Size'] = data['Outlet_Size'].replace(np.nan,'Medium')

In [ ]:
data.isnull().sum()

In [ ]:
sns.countplot(x='Item_Fat_Content',data=data)
plt.show()

In [ ]:
data['Item_Fat_Content'].replace('low fat','Low Fat',inplace=True)
data['Item_Fat_Content'].replace('LF','Low Fat',inplace=True)
data['Item_Fat_Content'].replace('reg','Regular',inplace=True)

In [ ]:
sns.countplot(x='Item_Fat_Content',data=data)
plt.show()

In [ ]:
sns.countplot(x='Outlet_Location_Type',data=data)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='Outlet_Type', data = data)
plt.show()

In [ ]:
plt.figure(figsize=(24,6))
sns.countplot(x='Item_Type',data=data)
plt.show()

In [ ]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()


In [ ]:
data.describe()

In [ ]:
data['Item_Visibility'] = data['Item_Visibility'].replace(0.0,data['Item_Visibility'].mean())


In [ ]:
#Mark non-consumables as separate category in low_fat:
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] ="Non-Edible"
data['Item_Fat_Content'].value_counts()


In [ ]:
sns.countplot(x='Item_Fat_Content', data = data)
plt.show()

In [ ]:
data.head()

In [ ]:
data.drop(['Item_Type','Item_Identifier'],axis=1,inplace=True)
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']

train.drop('source',axis=1,inplace=True)
test.drop(['source','Item_Outlet_Sales'],axis=1,inplace=True)

In [ ]:
train.shape, test.shape

In [ ]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_mod =['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet_Identifier']
le = LabelEncoder()
for i in var_mod:
    train[i] = le.fit_transform(train[i])
    test[i] = le.fit_transform(test[i])


In [ ]:
X = train.drop('Item_Outlet_Sales',axis=1)
y= train['Item_Outlet_Sales']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(X_train, y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

In [ ]:
mutual_info.sort_values(ascending=False).plot.bar(figsize=(15,5))
plt.show()

In [ ]:
from sklearn.feature_selection import SelectPercentile

## Selecting the top 40 percentile
selected_top_columns = SelectPercentile(mutual_info_regression, percentile=40)
selected_top_columns.fit(X_train, y_train)

In [ ]:
X_train.columns[selected_top_columns.get_support()]

In [ ]:
X_train.drop(['Item_Visibility','Item_Weight','Outlet_Location_Type','Item_Type_Combined','Item_Fat_Content','Outlet_Size'],axis=1)
X_test.drop(['Item_Visibility','Item_Weight','Outlet_Location_Type','Item_Type_Combined','Item_Fat_Content','Outlet_Size'],axis=1)

In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense

In [ ]:
input_neurons =X_train.shape[1]
output_neurons = 1

In [ ]:
number_of_hidden_layers = 2
neuron_hidden_layer_1 = 10
neuron_hidden_layer_2 = 5

In [ ]:
# defining the architecture of the model
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons,)))
model.add(Dense(units=neuron_hidden_layer_1, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_2,activation='relu'))
model.add(Dense(units=output_neurons, activation='linear'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
# training the model

# passing the independent and dependent features for training set for training the model

# validation data will be evaluated at the end of each epoch

# setting the epochs as 50

# storing the trained model in model_history variable which will be used to visualize the training process

model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,pred)

In [ ]:

# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(model_history.history['mean_absolute_error'])
plt.plot(model_history.history['val_mean_absolute_error'])
plt.title('model accuracy')
plt.ylabel('mean_absolute_error')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()